# Constraint Consistency Checker - Google Colab Setup

Competition-grade system using Qwen2.5-14B (4-bit), bge-m3, bge-reranker, and Pathway.

**Setup:** Runtime → Change runtime type → GPU (T4)

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Mount Drive or clone repo
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/IIT_KDSH

In [ ]:
# Install dependencies
!pip install -q torch transformers sentence-transformers bitsandbytes accelerate rank-bm25 tqdm

In [ ]:
# Verify setup
import torch
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Quick test
!python quickstart.py

In [ ]:
# Load models (first time: ~10GB download)
from models import load_llm, load_embedder, load_reranker

model, tokenizer = load_llm()
embedder = load_embedder()
reranker = load_reranker()

print(f"\nGPU memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

In [ ]:
# Test single example
from pipeline import ingest_novel, process_statement

sample_novel = """Chapter 1: Dr. John Smith worked at the hospital for 20 years.
Chapter 2: John treated patients daily.
Chapter 3: Lawyer John Smith argued his case in court."""

vector_store = ingest_novel(sample_novel, "test", embedder)

models = {'qwen': model, 'tokenizer': tokenizer, 'embedder': embedder, 'reranker': reranker}
result, evidence = process_statement(vector_store, models, "John Smith is a doctor.")

print(f"\nResult: {result} (0=violation, 1=consistent)")

In [ ]:
# Run full pipeline
!python run.py

In [ ]:
# View results
import pandas as pd
results = pd.read_csv("outputs/results.csv")
print(f"Total: {len(results)}")
results.head()

In [ ]:
# Calculate accuracy
if 'label' in results.columns:
    acc = (results['prediction'] == results['label']).mean()
    print(f"Accuracy: {acc:.2%}")

In [ ]:
# Download results
from google.colab import files
files.download('outputs/results.csv')